In [2]:
import os, sys, time
from time import sleep
from pathlib import Path

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [4]:
import logging

---
# *TABLE OF CONTENTS:*
---

### 1) Tools&Tricks
### 2) Error-Handling
### 3) DATETIME/TIMESTAMP
### 4) Logging
### 5) PANDAS
### 6) Concurrency:
- threading
- subprocess
- asyncio

### 7) Socket&Servers
### 8) Database/Connections
### 9)
### 10)

---
### *TO_DO:*
- [ ]  write table of contents
- [ ] flesh out document structure
- [ ] correct code errors
- [ ] finalize content

---
## `TOOLS&TRICKS`
---

### *ZIP files in directory and return .zip* **DESCRIPTIONS**
- #### changes working directory to allow creation of single tree zip folder
- #### changes directory back after zip is completed

In [ ]:
# INSTANTIATE GLOBAL VARIABLE / ORIGINAL WORKING DIRECTORY
og_wd = os.getcwd()

In [ ]:
def zip_the_directory(directory_to_zip): #{
    # RE-INSTANTIATE GLOBALS
    global og_wd
    # TRY THE FOLLOWING
    try: #{
        logging.info("\n\n\nXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        logging.info("\nWORKING DIRECTORY ---BEFORE--- ZIP==\n" + str(os.getcwd()))
        # CHANGE WORKING DIRECTORY TO DIRECTORY WE WISH TO ZIP
        os.chdir(directory_to_zip)
        # path to folder (NOW currently folder we want to zip)
        directory = "."
        logging.info("\nWORKING DIRECTORY ---DURING--- ZIP==\n" + str(os.getcwd()))
        # calling function to get all file paths in the directory
        file_paths = get_all_file_paths(directory)
        # printing the list of all files to be zipped 
        logging.info('\nFollowing files will be zipped:') 
        for file_name in file_paths: #{
            logging.info(file_name) 
        #}
        # writing files to a zipfile 
        with ZipFile('CofA-' 
                     + str(pd.Timestamp.now())[:10] 
                     + ".zip",'w') as zip: #{
            # writing each file one by one 
            for file in file_paths: #{
                zip.write(file) 
            #}
        #}
        logging.info('All files zipped successfully!\n\nXXXXXXXXXXXXXXXXXXXXXXXXX' 
                     + 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX\n')
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
                      "\n" + fileE +
                      "\n" + lineE +
                      "\n" + messageE)
    #}
    else: #{
        logging.info("[Zip-Files-" + str(directory_to_zip) + "] SUCCESS! VERY NICE!")
    #}
    finally: #{
        # CHANGE WORKING DIRECTORY BACK TO ORIGINAL
        os.chdir(og_wd)
    #}
#}

### *scan directory and return file* **DESCRIPTIONS**
- #### allowing choice of `directory`
- #### allowing choice of `directories_to_ignore`
- #### allowing choice of `search_file_types`

In [1]:
from os.path import join, getsize

### Option 1:

In [ ]:
# iterate thru directory/directories 
for root,d_names,f_names in os.walk(in_directory): #{
    print("===================== START ============================================== \n \n")
    print("\t *root* = " + str(root) + "\n") 
    print("\t *dict* = " + str(d_names) + "\n")
    if len(f_names) > 0: #{
        print("\t *file(s)*: \n")
        # multiple file(s)
        for f in f_names: #{
            # CHECK IF ROOT IS IN ONE OF THE DIRECTORIES WE WISH TO IGNORE!
            if str(root) == "G:\C of A's": #{
                pass
            #}
            elif str(root) == "G:\C of A's\#Email Node": #{
                pass
            #}
            elif str(root) == "G:\C of A's\OMNION": #{
                pass
            #}
            elif fnmatch.fnmatch(root, "G:\C of A's\OMNION\*"): #{
                pass
            #}
            # OTHERWISE... we are in a directory we want to be in
            else: #{
                # CHECK AND MAKE SURE ITS NOT A POOPY .db file
                if fnmatch.fnmatch(f, "*.db") is False: #{
                    the_str = str(f)
                    the_len = len(the_str)
                    # setup index
                    idx = the_str.rfind("@", 0, the_len)
                    # add (first half of file) to lists
                    part_no = f[0:idx]
                    part_list.append(part_no)
                    #full_path = os.path.join(d_names, f)
                    # add DIR NAME to lists
                    dir_list.append(root)
                    # add FILE NAME to lists
                    file_list.append(str(Path(f)))
                #}
            #}
        #}
    #}
    else: #{
        # no files
        print("\t NO-FILES!")
    #}
    print("\n \n================================================ END ===================== \n")
#}

### Option 2:

In [ ]:
"""
TAKES IN:
(1) FILE-PATH obj containing:
    - directory that needs to be scanned
(2) LIST-type obj containing:
    - list of STR (if any) of directories to skip/avoid
(3) LIST-type obj containing:
    - list of STR (more than one) of file type to return in scan
RETURNS:
DataFrame? of all scanned objects that might criteria etc
"""
def scan_directory(the_directory, ignore_dir_list, file_type_list): #{
    # counter
    x = 0
    # VARIABLES
    scan_directory = Path(the_directory)
    # create list to hold clean data yp
    path_list = []
    # BEGIN << OS.WALK >>
    for root, dirs, files in os.walk(scan_directory): #{
        #############################
        # DIRECTORY SKIP CONDITIONS #
        #############################
        # FOR EACH ITEM IN "ignore_dir_list":
        for item in ignore_dir_list: #{
            if str(item) in dirs: #{
                dirs.remove(str(item))  # REMOVE THAT FROM OS.WALK
            #}
        #}
        # OTHERWISE IF THERE ARE FILES IN DIRECTORY
        for f in files: #{
            ##############################
            # FILE MATCH-TYPE CONDITIONS #
            ##############################
            # FOR EACH ITEM IN "file_type_list":
            for item in file_type_list: #{
                # 
            #}
        #}
    #}
#}

>>>scan_directory(the_directory="F:/APPS/CofA/",             
               ignore_dir_list=["Archive ERR", "Archive - For all archived CofA, see G CofA folder", "Instruction Sheets"],
               file_type_list=[".pdf"]
               )

### *SCAN & MOVE files from directory to* **ANOTHER**
- #### allowing choice of `scan_directory`
- #### allowing choice of `destination_directory`
- #### allowing choice of `search_file_types`

In [1]:
def move_files(scan_directory, dst_directory, file_type_list): #{
    print("NO KILLER ALL FILLER")
#}

### *COPY/MOVE* files into a *TEMP* directory and then *ZIP* it

In [ ]:
import tempfile
with tempfile.TemporaryDirectory() as temporary_directory:  # {
    # COUNTER
    x1 = 0
    the_dir = Path(temporary_directory)
    # ITERATE THRU DATAFRAME
    for row in f_paths_df.itertuples(index=False, name='F_DRIVE'):  # {
        
    #}
#}

In [ ]:
def zip_the_directory(directory_to_zip): #{
    # RE-INSTANTIATE GLOBALS
    global og_wd
    # TRY THE FOLLOWING
    try: #{
        logging.info("\n\n\nXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        logging.info("\nWORKING DIRECTORY ---BEFORE--- ZIP==\n" + str(os.getcwd()))
        # CHANGE WORKING DIRECTORY TO DIRECTORY WE WISH TO ZIP
        os.chdir(directory_to_zip)
        # path to folder (NOW currently folder we want to zip)
        directory = "."
        logging.info("\nWORKING DIRECTORY ---DURING--- ZIP==\n" + str(os.getcwd()))
        # calling function to get all file paths in the directory
        file_paths = get_all_file_paths(directory)
        # printing the list of all files to be zipped 
        logging.info('\nFollowing files will be zipped:') 
        for file_name in file_paths: #{
            logging.info(file_name) 
        #}
        # writing files to a zipfile 
        with ZipFile('CofA-' 
                     + str(pd.Timestamp.now())[:10] 
                     + ".zip",'w') as zip: #{
            # writing each file one by one 
            for file in file_paths: #{
                zip.write(file) 
            #}
        #}
        logging.info('All files zipped successfully!\n\nXXXXXXXXXXXXXXXXXXXXXXXXX' 
                     + 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX\n')
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
                      "\n" + fileE +
                      "\n" + lineE +
                      "\n" + messageE)
    #}
    else: #{
        logging.info("[Zip-Files-" + str(directory_to_zip) + "] SUCCESS! VERY NICE!")
    #}
    finally: #{
        # CHANGE WORKING DIRECTORY BACK TO ORIGINAL
        os.chdir(og_wd)
    #}
#}

---
## `METADATA`
---

In [ ]:
"""
takes in two file path destinations,
and copies the METADATA from the old to new
"""
def old_file_metadata(old_path, new_path):  # {
    # GET METADATA OF OLD ORIGINAL FILE
    old_stinfo = os.stat(old_path)
    print("OLD FILE STATS: \n" + str(old_stinfo))
    old_atime = old_stinfo.st_atime
    print("OLD FILE A-TIME: \n" + str(old_atime))
    old_mtime = old_stinfo.st_mtime
    print("OLD FILE M-TIME: \n" + str(old_mtime))
    # CHANGE METADATA OF COPIED FILE TO ORIGINAL
    os.utime(new_path, (old_atime, old_mtime))
#}

---
## `FILLER`
---

---
## `EXCEPTION/ERROR-HANDLING`
---

In [ ]:
try: #{
    
#}
except: #{
    errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    typeE = str("TYPE : " + str(exc_type))
    fileE = str("FILE : " + str(fname))
    lineE = str("LINE : " + str(exc_tb.tb_lineno))
    messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
    logging.error("\n" + typeE +
                  "\n" + fileE +
                  "\n" + lineE +
                  "\n" + messageE)
#}
else: #{
    print("FIN...")
#}
finally: #{
    
#}

---
## `DATETIME/TIMESTAMP`
---

## 1

In [2]:
from datetime import datetime 

In [7]:
"""
TAKES IN:
(1) file_path to file 
RETURNS: 
The create time or modified time, whichever is older
"""
def pull_creation_timetamp(a_file_path): #{
    # FORCE PATH VARIABLE
    the_path = Path(a_file_path)
    # GET MODIFIED TIME
    mtime = os.path.getmtime(the_path)
    # GET CREATE TIME
    ctime = os.path.getctime(the_path)
    # CREATE DATE VAR
    # IF CREATE TIME IS OLDER...
    if ctime < mtime: #{
        # FORMAT DATE VAR as str
        date_str = str(datetime.fromtimestamp(ctime))
    #}
    # ELSE.... MODIFIED TIME IS OLDER...
    else: #{
        # FORMAT DATE VAR as str
        date_str = str(datetime.fromtimestamp(mtime))
    #}
    # RETURN THE DATE WE PULLED AS STRING
    return date_str
#}

In [10]:
test_path = "F:/APPS/CofA/PST-2280-10MG@0006484471.pdf"  # "F:/APPS/CofA/061496-1@NT056433.pdf"

In [11]:
print(pull_creation_timetamp(a_file_path=test_path))

2019-08-23 07:30:43.753058


## 2

In [1]:
def creation_date(path_to_file): # {
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    """
    if platform.system() == 'Windows': # {
        return os.path.getctime(path_to_file)
    # }
    else: # {
        stat = os.stat(path_to_file)
        try: # {
            return stat.st_birthtime
        # }
        except AttributeError: # {
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            return stat.st_mtime
        # }
    # }
    return
# }

## 3

*convert `pd.Timestamp` to `datetime`

In [2]:
ts_today = pd.Timestamp.now()
print(ts_today)

2019-09-04 12:05:10.983943


In [3]:
# CONVERT TO DATETIME
datetime_today = ts_today.to_datetime64()
print(datetime_today)

2019-09-04T12:05:10.983943000


---
## `TIME-SERIES (based)`
---

## 1 [timer]

In [1]:
from threading import Timer

In [2]:
def timer_test(): #{
    print("DING! ")
#}

In [3]:
# CREATE TIMER VARIABLE
t = Timer(interval=30, function=timer_test)

In [4]:
# START TIMER
t.start()

DING! 


---
## `LOGGING`
---

In [5]:
import logging

In [6]:
try: #{
    logging.basicConfig(level=logging.INFO,
                        filename='C:/CofA/log/CofA_Email_Node.log',
                        format='%(asctime)s:'
                        + '\n\t\t\t\t'
                        + '<MESG:%(message)s>'
                        + '\n\t\t\t\t'
                        + '<%(threadName)s-ID:%(thread)d>'
                        + '\n\t\t\t\t'
                        + '<%(processName)s-ID:%(process)d>'
                        + '\n\t\t\t\t'
                        + '<FUNC=%(funcName)s>'
                        + '\n\t\t\t\t'
                        + '<LINE:%(lineno)s>'
                        + '\n\t\t\t\t'
                        + '<PATH:%(pathname)s>'
                        + '\n\t\t\t\t'
                        + '<NAME:%(name)s>',
                        #format='%(asctime)s:\n\t\t\t\t%(message)s\n\t\t\t\t<%(threadName)s-ID:%(thread)d>\n<FUNC=%(funcName)s>\n<LINE:%(lineno)s>',
                        datefmt='%Y-%m-%d-%H%M%S',
                        filemode='a')
#}
except: #{
    errorMessage = str(sys.exc_info()[0]) + "\n"
    errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n"
    errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    typeE = str("TYPE : " + str(exc_type))
    fileE = str("FILE : " + str(fname))
    lineE = str("LINE : " + str(exc_tb.tb_lineno))
    messageE = str("MESG : " + "\n" + str(errorMessage))
    error_output = str("\n" + typeE +
                       "\n" + fileE +
                       "\n" + lineE +
                       "\n" + messageE)
    print(error_output)
#}
else: #{
    print("SUCCESS! VERY NICE!")
#}
finally: #{
    print("FIN...")
#}

SUCCESS! VERY NICE!
FIN...


---
# *`PANDAS`*
___

### *Preliminary Imports*

In [4]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [5]:
timestamp_now = pd.Timestamp.now()

In [6]:
print(timestamp_now)

2019-07-31 22:12:22.942243


---
# *`CONCURRENCY`*
---

---
#### `THREADING`

[threading-docs](https://docs.python.org/3.7/library/threading.html)
---

In [1]:
from threading import Timer

In [2]:
def hello(): #{
    print("hello world!")
#}

t = Timer(30.0, hello)
t.start() # after 30 seconds, "hello world will be printed"

hello world!


---
#### `SUBPROCESS`
---

---
#### `ASYNCIO`
---